In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import albumentations as A
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imgaug as iaa
import skimage.io
import keras.callbacks


# Root directory of the project
ROOT_DIR = os.path.abspath(os.getcwd())

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config


import Monuseg_dist

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
MONUSEG_DIR = os.path.join(ROOT_DIR, "datasets", "MoNuSeg_Dist")

# Comment out to reload imported modules if they change
%load_ext autoreload
%autoreload 2

c:\tensorlflow1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' 

In [2]:
# Set up augmentation, additional target for dist channel of image:
# (Treated as mask-type for the putposes of applying augmentations)
augmentation =  A.Compose([
    #A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, brightness_by_max=True, p=0.4),
#    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.1),
    A.HorizontalFlip(always_apply=False, p=0.5),
    A.VerticalFlip(always_apply=False, p=0.5),
    A.Rotate(limit = 359, always_apply=False, p=0.5),
], p=1,
    additional_targets={'dist': 'mask', 'Hch': 'mask'}
)

In [3]:
# Set up variables for training
config = Monuseg_dist.MonusegDIST_HConfig()
config.NAME = "MoNuSeg_Coco_Opttrain_DISTH_tissue"
config.VALIDATION_STEPS = 0
config.display()

DEVICE = "/gpu:0"
TEST_MODE = "training" #"inference" or "training"

dataset_dir = MONUSEG_DIR
subset = "train"
dataset_dir = os.path.join(dataset_dir, subset)
image_dir = os.path.join(dataset_dir, "tissue_images")
image_ids = os.listdir(image_dir)
print("Found {} images in {}".format(len(image_ids), image_dir))



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            5
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              crop
IMAGE_SHAPE                    [256 256   5]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.0001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE               

In [4]:
# Create the model, load weights
dataset_dir = MONUSEG_DIR
subset = "train"
dataset_dir = os.path.join(dataset_dir, subset)
image_dir = os.path.join(dataset_dir, "tissue_images")
image_ids = os.listdir(image_dir)
print("Loading data from: ", image_dir)

with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode=TEST_MODE, model_dir=MODEL_DIR, config=config, verbose = False)

#weights_path = model.find_last()
weights_path = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
#weights_path = r"C:\Users\User\GitHub\Mask_RCNN_Thesis\logs\pannuke model20210622T1625\mask_rcnn_pannuke model_0036.h5"


print("Loding model from: {}".format(weights_path))

#Exclude COCO heads
model.load_weights(weights_path, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
            "mrcnn_bbox", "mrcnn_mask"])

#model.load_weights(weights_path, by_name=True)
print("Model name: {}".format(config.NAME))

# Set up Datasets Objects 
dataset_train = Monuseg_dist.MonusegDISTDataset()
dataset_val = Monuseg_dist.MonusegDISTDataset()
dataset_train.add_class(source = "Monuseg", class_id =  1, class_name = "nucleus")
dataset_val.add_class(source = "Monuseg", class_id =  1, class_name = "nucleus")

# Fill with the samples
i = 0
for _id in image_ids:
    name,_ = os.path.splitext(_id)
    if i <= len(image_ids):
        dataset_train.add_image(source = "Monuseg",
                                image_id = name,
                                path = os.path.join(image_dir, _id))
    i+=1


dataset_val.add_image(source = "Monuseg",
                                image_id = name,
                                path = os.path.join(image_dir, _id))
    


dataset_val.prepare()
dataset_train.prepare()

print("Have {} train images".format(len(dataset_train.image_ids)))
print("Have {} val images".format(len(dataset_val.image_ids)))

Loading data from:  C:\Users\User\GitHub\Mask_RCNN_Thesis\datasets\MoNuSeg_Dist\train\tissue_images






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead
Loding model from: C:\Users\User\GitHub\Mask_RCNN_Thesis\mask_rcnn_coco.h5
Model name: MoNuSeg_Coco_Opttrain_DISTH_tissue
Have 30 train images
Have 1 val images


In [5]:
# Regular training
model.train(dataset_train, dataset_val,
       learning_rate=config.LEARNING_RATE,
        epochs=50,
        augmentation=augmentation,
        layers='all')


Starting at epoch 0. LR=0.0001

Checkpoint Path: C:\Users\User\GitHub\Mask_RCNN_Thesis\logs\monuseg_coco_nonopttrain_disth_tissue20210702T1732\mask_rcnn_monuseg_coco_nonopttrain_disth_tissue_{epoch:04d}.h5
Selecting layers to train
PREPROC                (Conv2D)
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm

c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




Epoch 1/50
60/60 [==============================] - 92s 2s/step - loss: 2.4560 - rpn_class_loss: 0.2824 - rpn_bbox_loss: 0.9614 - mrcnn_class_loss: 0.1587 - mrcnn_bbox_loss: 0.5517 - mrcnn_mask_loss: 0.5017 - val_loss: 2.2956 - val_rpn_class_loss: 0.3587 - val_rpn_bbox_loss: 0.7894 - val_mrcnn_class_loss: 0.3206 - val_mrcnn_bbox_loss: 0.4311 - val_mrcnn_mask_loss: 0.3957
Epoch 2/50
60/60 [==============================] - 71s 1s/step - loss: 1.9813 - rpn_class_loss: 0.1936 - rpn_bbox_loss: 0.8374 - mrcnn_class_loss: 0.1743 - mrcnn_bbox_loss: 0.4056 - mrcnn_mask_loss: 0.3703 - val_loss: 1.7534 - val_rpn_class_loss: 0.1686 - val_rpn_bbox_loss: 0.7581 - val_mrcnn_class_loss: 0.3165 - val_mrcnn_bbox_loss: 0.2317 - val_mrcnn_mask_loss: 0.2784
Epoch 3/50
60/60 [==============================] - 80s 1s/step - loss: 1.7517 - rpn_class_loss: 0.1586 - rpn_bbox_loss: 0.6624 - mrcnn_class_loss: 0.2517 - mrcnn_bbox_loss: 0.3445 - mrcnn_mask_loss: 0.3345 - val_loss: 1.5778 - val_rpn_class_loss: 0.

Epoch 22/50
60/60 [==============================] - 80s 1s/step - loss: 1.0070 - rpn_class_loss: 0.0669 - rpn_bbox_loss: 0.3542 - mrcnn_class_loss: 0.1936 - mrcnn_bbox_loss: 0.1792 - mrcnn_mask_loss: 0.2130 - val_loss: 0.9827 - val_rpn_class_loss: 0.1370 - val_rpn_bbox_loss: 0.3056 - val_mrcnn_class_loss: 0.1978 - val_mrcnn_bbox_loss: 0.1473 - val_mrcnn_mask_loss: 0.1949
Epoch 23/50
60/60 [==============================] - 80s 1s/step - loss: 1.0269 - rpn_class_loss: 0.0698 - rpn_bbox_loss: 0.3555 - mrcnn_class_loss: 0.1963 - mrcnn_bbox_loss: 0.1842 - mrcnn_mask_loss: 0.2211 - val_loss: 0.8165 - val_rpn_class_loss: 0.0863 - val_rpn_bbox_loss: 0.2170 - val_mrcnn_class_loss: 0.1824 - val_mrcnn_bbox_loss: 0.1284 - val_mrcnn_mask_loss: 0.2024
Epoch 24/50
60/60 [==============================] - 80s 1s/step - loss: 0.9253 - rpn_class_loss: 0.0644 - rpn_bbox_loss: 0.2934 - mrcnn_class_loss: 0.1916 - mrcnn_bbox_loss: 0.1631 - mrcnn_mask_loss: 0.2129 - val_loss: 0.6969 - val_rpn_class_loss: 0

Epoch 44/50
60/60 [==============================] - 81s 1s/step - loss: 0.8979 - rpn_class_loss: 0.0548 - rpn_bbox_loss: 0.2947 - mrcnn_class_loss: 0.1672 - mrcnn_bbox_loss: 0.1765 - mrcnn_mask_loss: 0.2047 - val_loss: 0.7448 - val_rpn_class_loss: 0.0567 - val_rpn_bbox_loss: 0.1600 - val_mrcnn_class_loss: 0.2279 - val_mrcnn_bbox_loss: 0.1172 - val_mrcnn_mask_loss: 0.1830
Epoch 45/50
60/60 [==============================] - 80s 1s/step - loss: 0.8209 - rpn_class_loss: 0.0489 - rpn_bbox_loss: 0.2741 - mrcnn_class_loss: 0.1643 - mrcnn_bbox_loss: 0.1540 - mrcnn_mask_loss: 0.1796 - val_loss: 0.9862 - val_rpn_class_loss: 0.1107 - val_rpn_bbox_loss: 0.4629 - val_mrcnn_class_loss: 0.1839 - val_mrcnn_bbox_loss: 0.0865 - val_mrcnn_mask_loss: 0.1422
Epoch 46/50
60/60 [==============================] - 78s 1s/step - loss: 0.8255 - rpn_class_loss: 0.0450 - rpn_bbox_loss: 0.2738 - mrcnn_class_loss: 0.1716 - mrcnn_bbox_loss: 0.1569 - mrcnn_mask_loss: 0.1782 - val_loss: 0.9765 - val_rpn_class_loss: 0

In [7]:
# Optimal stage-based training
model.train(dataset_train, dataset_val,
       learning_rate=config.LEARNING_RATE*2,
        epochs=5,
        augmentation=augmentation,
        layers=r"(PREPROC.*)|(conv1.*)|(mrcnn\_.*)")

model.train(dataset_train, dataset_val,
       learning_rate=config.LEARNING_RATE,
        epochs=5+35,
        augmentation=augmentation,
        layers="all")

model.train(dataset_train, dataset_val,
       learning_rate=config.LEARNING_RATE * 0.5,
        epochs=40+5,
        augmentation=augmentation,
        layers='5+')

model.train(dataset_train, dataset_val,
       learning_rate=config.LEARNING_RATE * 0.1,
        epochs=45+5,
        augmentation=augmentation,
        layers='heads')




Starting at epoch 0. LR=0.0002

Checkpoint Path: C:\Users\User\GitHub\Mask_RCNN_Thesis\logs\monuseg_coco_nonopttrain_disth_tissue20210702T1854\mask_rcnn_monuseg_coco_nonopttrain_disth_tissue_{epoch:04d}.h5
Selecting layers to train
PREPROC                (Conv2D)
conv1                  (Conv2D)
In model:  rpn_model
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)
mrcnn_class_logits     (TimeDistributed)
mrcnn_mask             (TimeDistributed)


c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node PREPROC_1/convolution}}]]
	 [[mul_67/_5305]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node PREPROC_1/convolution}}]]
0 successful operations.
0 derived errors ignored.

## Ensemble

In [5]:
dataset_dir = MONUSEG_DIR
subset = "train"
dataset_dir = os.path.join(dataset_dir, subset)
    
dict_names = {}

image_dir = os.path.join(dataset_dir, "tissue_images")
image_ids = os.listdir(image_dir)
# Fill with the samples
for _id in image_ids:
    name,_ = os.path.splitext(_id)
    dict_names[name] = os.path.join(image_dir, _id)

print(image_dir)
for k in dict_names.keys():
    print(k)

C:\Users\User\GitHub\Mask_RCNN_Thesis\datasets\MoNuSeg_Dist\train\tissue_images
TCGA-18-5592-01Z-00-DX1
TCGA-21-5784-01Z-00-DX1
TCGA-21-5786-01Z-00-DX1
TCGA-38-6178-01Z-00-DX1
TCGA-49-4488-01Z-00-DX1
TCGA-50-5931-01Z-00-DX1
TCGA-A7-A13E-01Z-00-DX1
TCGA-A7-A13F-01Z-00-DX1
TCGA-AR-A1AK-01Z-00-DX1
TCGA-AR-A1AS-01Z-00-DX1
TCGA-AY-A8YK-01A-01-TS1
TCGA-B0-5698-01Z-00-DX1
TCGA-B0-5710-01Z-00-DX1
TCGA-B0-5711-01Z-00-DX1
TCGA-CH-5767-01Z-00-DX1
TCGA-DK-A2I6-01A-01-TS1
TCGA-E2-A14V-01Z-00-DX1
TCGA-E2-A1B5-01Z-00-DX1
TCGA-G2-A2EK-01A-02-TSB
TCGA-G9-6336-01Z-00-DX1
TCGA-G9-6348-01Z-00-DX1
TCGA-G9-6356-01Z-00-DX1
TCGA-G9-6362-01Z-00-DX1
TCGA-G9-6363-01Z-00-DX1
TCGA-HE-7128-01Z-00-DX1
TCGA-HE-7129-01Z-00-DX1
TCGA-HE-7130-01Z-00-DX1
TCGA-KB-A93J-01A-01-TS1
TCGA-NH-A8F7-01A-01-TS1
TCGA-RD-A8N9-01A-01-TS1


In [6]:
# Mixture = 2x Bladder, Colon, Stomach
# 1: Kidney, Mixture, Liver, Prostate (No Breast)
# 2: Mixture, Liver, Prostate, Breast (No Kidney)
# 3: Liver, Prostate, Breast, Kidney (No Mixture)
# 4: Prostate, Breast, Kidney, Mixture (No Liver)
# 5: Breast, Kidney, Mixture, Liver (No Prostate) 

organ_dict = {
"Breast":["TCGA-A7-A13E-01Z-00-DX1",
    "TCGA-A7-A13F-01Z-00-DX1",
    "TCGA-AR-A1AK-01Z-00-DX1",
    "TCGA-AR-A1AS-01Z-00-DX1",
    "TCGA-E2-A14V-01Z-00-DX1",
    "TCGA-E2-A1B5-01Z-00-DX1"],
"Kidney":["TCGA-B0-5698-01Z-00-DX1",
    "TCGA-B0-5710-01Z-00-DX1",
    "TCGA-B0-5711-01Z-00-DX1",
    "TCGA-HE-7128-01Z-00-DX1",
    "TCGA-HE-7129-01Z-00-DX1",
    "TCGA-HE-7130-01Z-00-DX1"],
"Liver":["TCGA-18-5592-01Z-00-DX1",
    "TCGA-21-5784-01Z-00-DX1",
    "TCGA-21-5786-01Z-00-DX1",
    "TCGA-38-6178-01Z-00-DX1",
    "TCGA-49-4488-01Z-00-DX1",
    "TCGA-50-5931-01Z-00-DX1"],
"Prostate":["TCGA-CH-5767-01Z-00-DX1",
    "TCGA-G9-6336-01Z-00-DX1",
    "TCGA-G9-6348-01Z-00-DX1",
    "TCGA-G9-6356-01Z-00-DX1",
    "TCGA-G9-6362-01Z-00-DX1",
    "TCGA-G9-6363-01Z-00-DX1"],
"Mixture":["TCGA-KB-A93J-01A-01-TS1",
    "TCGA-RD-A8N9-01A-01-TS1",
    "TCGA-AY-A8YK-01A-01-TS1",
    "TCGA-NH-A8F7-01A-01-TS1",
    "TCGA-DK-A2I6-01A-01-TS1",
    "TCGA-G2-A2EK-01A-02-TSB"]
}

fold1 = []
for k in ["Kidney", "Liver", "Prostate", "Mixture"]:
    fold1.extend(organ_dict[k])

fold2 = []
for k in ["Mixture", "Liver", "Prostate", "Breast"]:
    fold2.extend(organ_dict[k])

fold3 = []
for k in ["Liver", "Prostate", "Breast", "Kidney"]:
    fold3.extend(organ_dict[k])
    
fold4 = []
for k in ["Prostate", "Breast", "Kidney", "Mixture"]:
    fold4.extend(organ_dict[k])
    
fold5 = []
for k in ["Breast", "Kidney", "Mixture", "Liver"]:
    fold5.extend(organ_dict[k])
    
       
fold_list = [fold1, fold2, fold3, fold4, fold5]
for f in fold_list:
    print(len(f))

24
24
24
24
24


In [7]:
config = Monuseg_dist.MonusegDIST_HConfig()
config.VALIDATION_STEPS = 0
i = 0

for fold in fold_list:
    # Used to choose model fold number
    if i >= 2:   
        print("Model Nr. {} Fold:".format(i))
        print(fold)
        dataset_train = Monuseg_dist.MonusegDISTDataset()
        dataset_val = Monuseg_dist.MonusegDISTDataset()
        dataset_train.add_class("Monuseg", 1, "nucleus")
        # Fill with the train samples
        for _n in fold:
            dataset_train.add_image(source = "Monuseg", image_id = _n, path = dict_names[_n])
        dataset_train.prepare()
        
        config.NAME = "Monuseg_5Fold_COCO_DIST_H_TISSUE_IMG" + str(i)
        
        with tf.device(DEVICE):
            model = modellib.MaskRCNN(mode=TEST_MODE, model_dir=MODEL_DIR, config=config, verbose = False)
        #wp = model.find_last()
        wp = COCO_PATH
        #wp = r"C:\Users\User\GitHub\Mask_RCNN_Thesis\logs\pannuke model20210622T1625\mask_rcnn_pannuke model_0036.h5"
        
        print("Loding model from: {}".format(wp))
        model.load_weights(wp, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                "mrcnn_bbox", "mrcnn_mask"])        
        #model.load_weights(wp, by_name = True)
        
        
        #n_epoch = 50
        #print("Train all layers")
        #model.train(dataset_train, dataset_val,
        #       learning_rate=config.LEARNING_RATE,
        #        epochs=n_epoch,
        #        augmentation=augmentation,
        #        layers='all')
        
        model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE*2,
            epochs=5,
            augmentation=augmentation,
            layers=r"(PREPROC)|(conv1.*)|(mrcnn\_.*)")

        model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=35+5,
            augmentation=augmentation,
            layers="all")

        model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE * 0.5,
            epochs=40+5,
            augmentation=augmentation,
            layers='5+')

        model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE * 0.1,
            epochs=40+5,
            augmentation=augmentation,
            layers='heads')



        del model
    i = i+1        
  

Model Nr. 2 Fold:
['TCGA-18-5592-01Z-00-DX1', 'TCGA-21-5784-01Z-00-DX1', 'TCGA-21-5786-01Z-00-DX1', 'TCGA-38-6178-01Z-00-DX1', 'TCGA-49-4488-01Z-00-DX1', 'TCGA-50-5931-01Z-00-DX1', 'TCGA-CH-5767-01Z-00-DX1', 'TCGA-G9-6336-01Z-00-DX1', 'TCGA-G9-6348-01Z-00-DX1', 'TCGA-G9-6356-01Z-00-DX1', 'TCGA-G9-6362-01Z-00-DX1', 'TCGA-G9-6363-01Z-00-DX1', 'TCGA-A7-A13E-01Z-00-DX1', 'TCGA-A7-A13F-01Z-00-DX1', 'TCGA-AR-A1AK-01Z-00-DX1', 'TCGA-AR-A1AS-01Z-00-DX1', 'TCGA-E2-A14V-01Z-00-DX1', 'TCGA-E2-A1B5-01Z-00-DX1', 'TCGA-B0-5698-01Z-00-DX1', 'TCGA-B0-5710-01Z-00-DX1', 'TCGA-B0-5711-01Z-00-DX1', 'TCGA-HE-7128-01Z-00-DX1', 'TCGA-HE-7129-01Z-00-DX1', 'TCGA-HE-7130-01Z-00-DX1']
Loding model from: C:\Users\User\GitHub\Mask_RCNN_Thesis\mask_rcnn_coco.h5

Starting at epoch 0. LR=0.0002

Checkpoint Path: C:\Users\User\GitHub\Mask_RCNN_Thesis\logs\monuseg_5fold_coco_dist_h_tissue_img220210703T1151\mask_rcnn_monuseg_5fold_coco_dist_h_tissue_img2_{epoch:04d}.h5
Selecting layers to train
PREPROC                (C

c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




Epoch 1/5
60/60 [==============================] - 89s 1s/step - loss: 3.2722 - rpn_class_loss: 0.7931 - rpn_bbox_loss: 1.1649 - mrcnn_class_loss: 0.0959 - mrcnn_bbox_loss: 0.6644 - mrcnn_mask_loss: 0.5539
Epoch 2/5


c:\tensorlflow1.14\lib\site-packages\keras\callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


60/60 [==============================] - 72s 1s/step - loss: 2.8297 - rpn_class_loss: 0.6888 - rpn_bbox_loss: 1.1042 - mrcnn_class_loss: 0.0443 - mrcnn_bbox_loss: 0.5122 - mrcnn_mask_loss: 0.4800
Epoch 3/5
60/60 [==============================] - 82s 1s/step - loss: 2.6182 - rpn_class_loss: 0.6905 - rpn_bbox_loss: 1.0782 - mrcnn_class_loss: 0.0341 - mrcnn_bbox_loss: 0.4248 - mrcnn_mask_loss: 0.3906
Epoch 4/5
60/60 [==============================] - 79s 1s/step - loss: 2.5778 - rpn_class_loss: 0.6781 - rpn_bbox_loss: 1.0646 - mrcnn_class_loss: 0.0299 - mrcnn_bbox_loss: 0.4331 - mrcnn_mask_loss: 0.3721
Epoch 5/5
60/60 [==============================] - 81s 1s/step - loss: 2.4940 - rpn_class_loss: 0.6535 - rpn_bbox_loss: 1.0680 - mrcnn_class_loss: 0.0300 - mrcnn_bbox_loss: 0.3953 - mrcnn_mask_loss: 0.3472

Starting at epoch 5. LR=0.0001

Checkpoint Path: C:\Users\User\GitHub\Mask_RCNN_Thesis\logs\monuseg_5fold_coco_dist_h_tissue_img220210703T1151\mask_rcnn_monuseg_5fold_coco_dist_h_tissue

...

c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
c:\tensorlflow1.14\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
60/60 [==============================] - 85s 1s/step - loss: 3.6964 - rpn_class_loss: 0.9474 - rpn_bbox_loss: 1.4498 - mrcnn_class_loss: 0.0633 - mrcnn_bbox_loss: 0.7120 - mrcnn_mask_loss: 0.5239
Epoch 2/5


c:\tensorlflow1.14\lib\site-packages\keras\callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


28/60 [=============>................] - ETA: 45s - loss: 3.1556 - rpn_class_loss: 0.8786 - rpn_bbox_loss: 1.1941 - mrcnn_class_loss: 0.0497 - mrcnn_bbox_loss: 0.5846 - mrcnn_mask_loss: 0.4485

ResourceExhaustedError: OOM when allocating tensor with shape[600,256,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_4/Adam/gradients/mrcnn_mask_2/convolution_grad/Conv2DBackpropInput}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
